# Libraries

In [3]:
import numpy as np
import pandas as pd
import time

import requests
from bs4 import BeautifulSoup

import sqlite3
from sqlalchemy import create_engine

# Functions

In [23]:
def extract_books(api_key, sleep):
    book_list = []

    payload = {'api-key': api_key,
               'offset': ''}
    url = 'https://api.nytimes.com/svc/books/v3/lists/best-sellers/history.json'
    resp_num = requests.get(url, params=payload).json()
    num_results = resp_num.get('num_results')
    offset_list = range(20, num_results - (num_results%20), 20)
    time.sleep(sleep)
    for offset_index in offset_list:
        resp = requests.get(url, params=payload).json()
        book_list.extend(resp.get('results'))
        payload.update({'offset': offset_index})
        time.sleep(sleep)
        
    return book_list

def flatten(books):
    for i in range(len(books)):
        if books[i].get('isbns') == []:
            books[i].update(books[i].get('isbns'))
        else:
            books[i].update(books[i].get('isbns')[0])
        if books[i].get('ranks_history') == []:
            books[i].update(books[i].get('ranks_history'))
        else:
            books[i].update(books[i].get('ranks_history')[0])
        books[i].update(books[i].get('reviews')[0])
        del books[i]['isbns']
        del books[i]['ranks_history']
        del books[i]['reviews']
        
    return books

def create_db(database):
    conn = sqlite3.connect(f'/mnt/processed/private/msds2023/lt9/{database}')
    gen_df.to_sql(name='gen_df', con=conn, if_exists='replace', index=False)
    conn.close()
    return None

# API CALLS

In [210]:
books = extract_books('m4KI3c95jMS0rloPbGNGtCiREBAv8Hi0', 6)

In [217]:
nyt_books = books.copy()

# Table Creation

In [218]:
book_table = flatten(nyt_books)
df_books = pd.DataFrame(book_table)

# SQLite Database Creation

In [224]:
create_db('books.db')

In [225]:
!ls /mnt/processed/private/msds2023/lt9

books.db  lfb.db


# Additional Tables

In [8]:
dates = pd.date_range(start='2008/1/1', end='10/31/2022')

In [9]:
dates_df = pd.DataFrame()
dates_df['dates'] = dates
dates_df['day_of_week'] = dates_df['dates'].dt.day_name()
dates_df = dates_df.loc[dates_df['day_of_week'] == 'Sunday']
sunday_list = dates_df['dates'].astype(str).tolist()

In [10]:
api_key = 'm4KI3c95jMS0rloPbGNGtCiREBAv8Hi0'
week_str = '2022-10-30'
def extract_overview(api_key, sleep, date_list):
    book_list = []
    for week_str in date_list:
        payload = {'api-key': api_key,
                   'published_date': week_str}
        url = 'https://api.nytimes.com/svc/books/v3/lists/overview.json'
        resp = requests.get(url, params=payload).json()
        book_list.append(resp.get('results'))
        time.sleep(sleep)
    return book_list

In [192]:
list_of_books = extract_overview('m4KI3c95jMS0rloPbGNGtCiREBAv8Hi0', 6, sunday_list)

In [214]:
books_with_url = list_of_books.copy()
books_with_url = books_with_url[22:]

In [222]:
df_1 = pd.DataFrame()
for i in books_with_url:
    df_0 = pd.DataFrame(i['lists'])
    df_1 = pd.concat([df_1, df_0])

In [279]:
books_list = df_1[['list_id', 'books']].copy()
new_dict = list(zip(books_list['list_id'], books_list['books']))
gen_df = pd.DataFrame()
for k, v in new_dict:
    df = pd.DataFrame(v)
    df['list_id'] = k
    gen_df = pd.concat([gen_df, df])

In [289]:
gen_df = gen_df.drop('buy_links', axis=1)

In [24]:
create_db('nytbooks.db')

In [25]:
!ls /mnt/processed/private/msds2023/lt9

books.db  fide.db  lfb.db  nytbooks.db


In [7]:
!ls /mnt/processed/private/msds2023/lt7

cali_traffic_coll.zip  switrs.sqlite
